<a href="https://colab.research.google.com/github/WiemHAD/clustering/blob/master/clustering2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Applying Kmeans algorith to emotion data set

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#global variables
df = pd.read_csv('emotion.csv')
stopwords = nltk.corpus.stopwords.words("english")
targets = df['Emotion']
corpus = df['Text']
# count vectorizer using SKlearn
vect = CountVectorizer(stop_words = stopwords)
X = vect.fit_transform(corpus)
words = vect.get_feature_names()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(targets)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 6)
kmeans_fitted = kmeans.fit_transform(X)

In [ ]:
from sklearn.metrics.cluster import adjusted_mutual_info_score, homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score
print('AMI score = ', adjusted_mutual_info_score(kmeans.labels_,targets))
print('homogeneity score =', homogeneity_score(kmeans.labels_,targets))
print('completeness_score = ',completeness_score(kmeans.labels_,targets))
print('V measure score =', v_measure_score(kmeans.labels_,targets))
print('Adjusted Rand Score =', adjusted_rand_score(kmeans.labels_, targets))

AMI score =  0.007869351582768509
homogeneity score = 0.009155314711364874
completeness_score =  0.007541844560975775
V measure score = 0.00827062368319247
Adjusted Rand Score = 0.010184088402371496


In [ ]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(with_mean=False), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean")
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=6,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X, labels=targets)

kmeans = KMeans(init="random", n_clusters=6,random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X, labels=targets)


clf = TruncatedSVD()
Xpca = clf.fit_transform(X)
XX = PCA(2).fit_transform(Xpca)
kmeans = KMeans(n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=XX, labels=targets)
print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	8.258s	408284055	0.000	0.119	0.000	-0.000	-0.000	0.200
random   	13.563s	408847945	0.002	0.110	0.004	-0.001	0.003	0.007
PCA-based	0.060s	6440	0.007	0.008	0.008	0.010	0.008	0.642
__________________________________________________________________________________


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(corpus)

In [ ]:
print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
kmeans = KMeans(init="k-means++", n_clusters=6, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X_tfidf, labels=y)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X_tfidf, labels=y)

clf = TruncatedSVD()
Xpca = clf.fit_transform(X)
XX_tfidf = PCA(2).fit_transform(Xpca)
kmeans = KMeans(n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=XX_tfidf, labels=y)
print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	11.775s	411236170	0.003	0.152	0.006	-0.002	0.005	-0.007
random   	6.429s	412060564	0.000	0.073	0.001	-0.000	0.000	-0.009
PCA-based	0.046s	4532	0.007	0.008	0.007	0.010	0.007	0.675
__________________________________________________________________________________


* Est-que les clusters obtenue avec kmeans reflete les groupements de texte par émotions ?

au vu des résultats obtenus, les clusters obtenus ne refléte pas les groupements de texte par émotions.

* Quelle est le nombre de cluster optimal au regard de la métrique "silhouette" ? de l'inertie ?  est ce que l'inertie est corrélé à la mesure de silhouette ?

on observe, sur les différents tableaux de résultats que:
- l'inertie est inversement proportionnelle aux autres metriques. 
- en combinant le TF-IDF, Truncated SVD et PCA, on obtient des meilleurs résultats. 
- le TF-IDF a permi d'améliorer la silhouette du random et du PCA mais celle du kmeans++ s'est dégradée.
- étant donnée que la metrique silhouette n'est pas révélatrice, le nombre de cluster doit être plus important
